In [8]:
import numpy as np
import os
import pickle

from scipy.misc import imread
from skimage.feature import hog

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [9]:
from lessons_functions import *

In [10]:
from sklearn import metrics
import pandas as pd

def confusion_matrix(true_y, pred_y, labels):
    c_matrix = metrics.confusion_matrix(true_y, pred_y)

    confusion_table = []
    first_row = ["C.Matrix"] + labels + ["ACTUAL"] + ["RECALL"]
    confusion_table.append(first_row)

    recall = metrics.recall_score(true_y, pred_y, average=None)
    for r, row in enumerate(c_matrix):
        new_row = [labels[r]]
        new_row.extend(row)
        new_row.append(sum(row))
        new_row.append(recall[r])
        confusion_table.append(new_row)

    new_row = ["PREDICTED"]
    for l in labels:
        new_row.append(len([t for t in pred_y if t == l]))
    new_row.append(len(true_y))
    new_row.append(metrics.recall_score(true_y, pred_y, average='macro'))
    confusion_table.append(new_row)

    new_row = ["PRECISION"]
    new_row.extend(metrics.precision_score(true_y, pred_y, average=None))
    new_row.append(metrics.precision_score(true_y, pred_y, average='macro'))
    new_row.append(metrics.f1_score(true_y, pred_y, average='macro'))
    confusion_table.append(new_row)

    confusion_table = pd.DataFrame(confusion_table)
    return confusion_table

In [11]:
def get_dataset():
    X = []
    y = []

    for root, dirs, files in os.walk("dataset/vehicles"):
        for file in files:
            if file.endswith(".png"):
                X.append(imread(os.path.join(root, file)))
                y.append(1)
            
    for root, dirs, files in os.walk("dataset/non-vehicles"):
        for file in files:
            if file.endswith(".png"):
                X.append(imread(os.path.join(root, file)))
                y.append(0)
                
    return np.asarray(X), np.asarray(y)

In [12]:
def get_features(img, orient=9, pix_per_cell=8, cell_per_block=2, spatial_size=(16, 16), hist_bins=32):
    
    # convert image into desired format
    img = img.astype(np.float32) / 255
    img = convert_color(img, conv='RGB2YCrCb')
    
    # extract channels
    ch1 = img[:,:,0]
    ch2 = img[:,:,1]
    ch3 = img[:,:,2]
                    
    # extract hog features
    hog1 = get_hog_features(
        ch1, orient, pix_per_cell, cell_per_block, feature_vec=False).ravel()
    hog2 = get_hog_features(
        ch2, orient, pix_per_cell, cell_per_block, feature_vec=False).ravel()
    hog3 = get_hog_features(
        ch3, orient, pix_per_cell, cell_per_block, feature_vec=False).ravel()
    hog_features = np.hstack((hog1, hog2, hog3))
    
    # extract spatial features  
    spatial_features = bin_spatial(img, size=spatial_size)
    
    # extract color histogram features
    hist_features = color_hist(img, nbins=hist_bins)
    
    # stack extracted features
    features = np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1)
    return features

In [13]:
X, y = get_dataset()

# vstack is slow, therefore I create a list, which is then converted into numpy array
features = []
for x in X:
    features.append(get_features(x))
features = np.asarray(features).squeeze()
print("Features shape: {}".format(features.shape))

ss = StandardScaler()
features = ss.fit_transform(features)

# split into training/validation set
X_train, X_valid, y_train, y_valid = train_test_split(
    features, y, test_size=0.33, random_state=42)

clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)

confusion_matrix(y_valid, clf.predict(X_valid), [0, 1])

Features shape: (17760, 6156)


,0,1,2,3,4
0,C.Matrix,0.000000,1.000000,ACTUAL,RECALL
1,0,2968.000000,6.000000,2974,0.997983
2,1,32.000000,2855.000000,2887,0.988916
3,PREDICTED,3000.000000,2861.000000,5861,0.993449
4,PRECISION,0.989333,0.997903,0.993618,0.993514


In [14]:
# save
pickle.dump(clf, open("svc_pickle.p", 'wb'))
pickle.dump(ss, open("scaler_pickle.p", 'wb'))